In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fsolve
from scipy.optimize import root

In [2]:
# import varibles in baseline period

N = 30
J = 2

## construction lands in urban sectors in 2010
K = np.array(pd.read_excel('./data/raw_and_build/urban_constructive_land_area.xlsx').loc[:,'Land2010'])
K = np.delete(K, 25).reshape([N, 1])

## trade share
pi = np.array(pd.read_excel('./data/raw_and_build/trade_share/trade_share_sep.xlsx').iloc[:, 1:]) 

## migration share
m = np.array(pd.read_excel('./data/raw_and_build/migration_share/migration_share_TZ2019.xlsx').iloc[:, 1:]).T

## total residents and population with local hukou
L_bar = np.array(pd.read_excel('./data/raw_and_build/macro_var_2010.xlsx').loc[:, 'L_bar']).reshape([N*J, 1])
L = m @ L_bar 

## nominal inocme and price index
w = np.array(pd.read_excel('./data/raw_and_build/macro_var_2010.xlsx').loc[:, 'w']).reshape([N*J, 1])
P = np.array(pd.read_excel('./data/raw_and_build/macro_var_2010.xlsx').loc[:, 'P']).reshape([N*J, 1])
W = w / P

In [3]:
# import parameter

kappa = 1.5 # migration elasticity, based on Tombe and Zhu (2019)
theta = 4 # trade elasticity, based on Tombe and Zhu (2019)
beta = 0.9 # consumption share, based on 2011 China Statistic Book
psi_ag, psi_na = 0.08, 0.92 # share on ag. and non-ag. products, based on I-O table in 2010
alpha_ag, alpha_na = 0.28, 0.15 # labor share, based on I-O table in 2010
sigma_ag, sigma_na = 0.30, 0.02 # land share, based on I-O table in 2010
eta_ag_ag, eta_ag_na = 0.13, 0.05 # ag. intermediates share, based on I-O table in 2010.
eta_na_ag, eta_na_na = 0.29, 0.78 # non-ag. intermediates share, based on I-O table in 2010.
phiu = 0.1 # externality of public expenditures on utility, based on Leeper et al.(2010)
phip = 0.05 # externality of public expenditures on production, based on Leeper et al. (2010)
chi = 0.9 # degree of rivalness of public goods, based on ZC2010
eta = 5 # elasticity of substitution of intermediates, arbitrage value
delta = 0 # degree of agglomeration effects, set to 0 in the baseline model
xi_ag, xi_na = 0.01, 0.99 # share on ag. and non-ag. products of government purchases, based on 2010 I-O table
t = 0.1 # tax rate

In [4]:
# Solve the baseline equilibrium (1)

R = np.ndarray([60, 1])
X0 = np.zeros([60, 1])
X = np.ones([60, 1])
iter_num = 0

while abs((X0 - X)).sum() > 1e-10:
    X0 = X
    R = pi @ X0
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    wL = wL / wL.sum() # standardize
    vL = (wL.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG = (wL.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A = (beta * (1 - t) * np.repeat(vL.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C = (np.repeat(PG.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag = ((R.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na = ((R.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B = np.concatenate([B_ag, B_na], axis=1).reshape([60, 1])
    X = A + B + C

    iter_num += 1
    print(str(iter_num), end='\r')

equilibrium_dict = {
    'pi' : pi,
    'm' : m,
    'L' : L,
    'L_bar' : L_bar,
    'w' : w,
    'P' : P,
    'W' : W,
    'R' : R,
    'X' : X,
    'K' : K
}

parameter = {
    'kappa' : kappa,
    'theta' : theta,
    'beta' : beta,
    'psi_ag' : psi_ag,
    'psi_na' : psi_na,
    'alpha_ag' : alpha_ag,
    'alpha_na' : alpha_na,
    'sigma_ag' : sigma_ag,
    'sigma_na' : sigma_na,
    'eta_ag_ag' : eta_ag_ag, 
    'eta_ag_na' : eta_ag_na,
    'eta_na_ag' : eta_na_ag, 
    'eta_na_na' : eta_na_na,
    'phiu' : phiu,
    'phip' : phip,
    'chi' : chi,
    'eta' : eta,
    'delta' : delta,
    'xi_ag' : xi_ag,
    'xi_na' : xi_na,
    't' : t
}
np.savez('./data/Equilibrium2010.npz', **equilibrium_dict)
np.savez('./data/Parameter.npz', **parameter)

In [5]:
# Solve the baseline equilibrium (2): use "root" to check

def SolveCounter(X):
    X = X.reshape([60, 1])
    R = pi @ X
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    wL = wL / wL.sum() # standardize
    vL = (wL.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG = (wL.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A = (beta * (1 - t) * np.repeat(vL.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C = (np.repeat(PG.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag = ((R.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na = ((R.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B = np.concatenate([B_ag, B_na], axis=1).reshape([60, 1])
    
    return (X - A - B - C).flatten()

X_test = root(SolveCounter, x0=np.ones(60))
X_test = X_test.x.reshape([60, 1])
print(abs(X - X_test).sum() < 0.000001) # check

True
